In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install contractions

In [ ]:

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

In [ ]:
import nltk
import re
import pickle
import numpy as np
import pandas as pd
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
df_2 = pd.read_csv("/kaggle/input/fake-news/train.csv", header=0, index_col=0)
df_t = pd.read_csv("/kaggle/input/fake-news/test.csv", header=0, index_col=0)
df_2 = df_2.drop(['title','author'], axis = 1)
df_t = df_t.drop(['title','author'], axis = 1)
df_2.dropna(inplace = True)
df_t.fillna('',inplace = True)
#print(df_2.isnull().sum(axis = 0))

#Expanding Contraction, Lower Case and Word Splitting
df_2['text'] = df_2['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])
df_t['text'] = df_t['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])

#Removing Punctuations
df_2['text'] = df_2['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])
df_t['text'] = df_t['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])

#Removing Stop Words
stop_words = set(stopwords.words('english'))
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word not in stop_words])
df_t['text'] = df_t['text'].apply(lambda x: [word for word in x if word not in stop_words])

#Removing Special Charecter and Numbers
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])
df_t['text'] = df_t['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])

#Removing Non-English Words
#df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word in english_words])

#Concating Words back to Sentence
#df_2['text'] = df_2['text'].apply(lambda x: ' '.join(x))

#Transforming Labels
df_2['label'] = df_2['label'].apply(lambda x: int(x))

print(df_2.head(3))

In [ ]:
df_2.head(2)

In [ ]:
df_t.isnull().sum()

In [ ]:
print(stop_words)

In [ ]:
#stemming 
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
x = []
x_test = []
english_words = set(nltk.corpus.words.words())
for words in df_2['text']:
        tmp = []
        #tmp1 = []
        fil_wor = [wordnet_lemmatizer.lemmatize(word, 'n') for word in words if word in english_words]
        #fil_wor1 = [wordnetlemmatizer.lemmatize(word) for word in words if word in english_words]
        tmp.extend(fil_wor)
        #tmp1.extend(fil_wor1)
        x.append(tmp)
        #x1.append(tmp1)
        
for words in df_t['text']:
        tmp = []
        #tmp1 = []
        fil_wor = [wordnet_lemmatizer.lemmatize(word, 'n') for word in words if word in english_words]
        #fil_wor1 = [wordnetlemmatizer.lemmatize(word) for word in words if word in english_words]
        tmp.extend(fil_wor)
        #tmp1.extend(fil_wor1)
        x_test.append(tmp)
        #x1.append(tmp1)


In [ ]:
df_2.shape

In [ ]:
len(x_test)

In [ ]:
df_t.shape

In [ ]:
 
x_all=  x.copy()


In [ ]:
#range(len(x_test))

In [ ]:
len(x_all)

In [ ]:
for y in  range(len(x_test)):
    x_all.append(x_test[y])

In [ ]:
len(x_all)

In [ ]:
import gensim

In [ ]:
#n of vectors we are generating
EMBEDDING_DIM = 100

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=x_all, vector_size=EMBEDDING_DIM, window=5, min_count=1)

In [ ]:
len(w2v_model.wv)

In [ ]:
w2v_model.wv["liberty"]

In [ ]:
word = 'people'
w2v_model.wv.most_similar(word)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)

In [ ]:
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
x[0][:10]

In [ ]:
word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break

In [ ]:
maxlen = 700
x = pad_sequences(x, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
y = df_2['label']

In [ ]:
#word_index

In [ ]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix, vocab_size

In [ ]:
#Getting embedding vectors from word2vec and usings it as weights of non-trainable keras embedding layer
embedding_vectors, vocab_size = get_weight_matrix(w2v_model.wv, word_index)

In [ ]:
vocab_size

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, Dropout

from tensorflow.keras.layers import Dropout, Conv1D, MaxPooling1D, BatchNormalization

In [ ]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(Dropout(0.2))
#model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=128,dropout=0.2, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=128,dropout=0.2))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

#del embedding_vectors

In [ ]:

model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [ ]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50)

In [ ]:

#Prediction is in probability of news being real, so converting into classes
# Class 0 (Fake) if predicted prob < 0.5, else class 1 (Real)
y_pred = (model.predict(X_test) >= 0.5).astype("int")

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred)) 

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
y_t = (model.predict(x_test) >= 0.5).astype("int")

In [ ]:
len(y_t.squeeze())

In [ ]:
len(df_t.index)

In [ ]:
pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [ ]:
result = pd.DataFrame({"id" :df_t.index, "label":y_t.squeeze() }, index = None )

In [ ]:
result.reset_index(drop = True, inplace = True)

In [ ]:
result.to_csv("result_2.csv",index = False)

In [ ]:
pd.read_csv("result_2.csv")